In [2]:
!pip install optuna
import json
import joblib
import optuna
import numpy as np
import pandas as pd
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from optuna.visualization import plot_optimization_history, plot_param_importances


   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   ---------------------------------------- 2/2 [optuna]



In [3]:
!pip install streamlit
!pip install shap

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -----------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\mailn\\AppData\\Local\\Temp\\pip-unpack-sw28du1q\\numpy-2.1.3-cp313-cp313-win_amd64.whl'
Consider using the `--user` option or check the permissions.



In [5]:
train = pd.read_csv("C:/Users/mailn/Downloads/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("C:/Users/mailn/Downloads/house-prices-advanced-regression-techniques/test.csv")

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,TotalSF
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890,2567.048630
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883,821.714421
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000,334.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000,2009.500000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000,2474.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000,3004.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000,11752.000000


In [12]:
train.isnull().sum() 

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
TotalSF            0
Length: 82, dtype: int64

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

In [15]:
# --- Feature engineering
train["TotalSF"] = (
    train["TotalBsmtSF"].fillna(0)
    + train["1stFlrSF"].fillna(0)
    + train["2ndFlrSF"].fillna(0)
)
y = train["SalePrice"]
X = train.drop(columns=["SalePrice", "Id"])

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

numeric = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric, num_cols),
    ("cat", categorical, cat_cols)
])



In [16]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,TotalSF
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890,2567.048630
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883,821.714421
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000,334.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000,2009.500000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000,2474.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000,3004.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000,11752.000000


In [26]:
# --- Optuna objective
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 800),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
    }
    model = XGBRegressor(**params)
    pipe = Pipeline([("pre", preprocessor), ("model", model)])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_val)
    return mean_squared_error(y_val, preds ) ** 0.5
    
    # --- Optuna objective

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)
print("✅ Best Parameters:", study.best_trial.params)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)
print("✅ Best Parameters:", study.best_trial.params)

[I 2025-10-29 05:51:21,166] A new study created in memory with name: no-name-3f7a8316-4b5f-4133-8a0d-4f543060fd60
[I 2025-10-29 05:51:21,939] Trial 0 finished with value: 25518.7422887571 and parameters: {'n_estimators': 336, 'learning_rate': 0.022458034230878686, 'max_depth': 5, 'subsample': 0.7673187787473119, 'colsample_bytree': 0.871876986403185}. Best is trial 0 with value: 25518.7422887571.
[I 2025-10-29 05:51:22,905] Trial 1 finished with value: 27294.240271529816 and parameters: {'n_estimators': 475, 'learning_rate': 0.12367982804598726, 'max_depth': 5, 'subsample': 0.9802937031735993, 'colsample_bytree': 0.8485408906642857}. Best is trial 0 with value: 25518.7422887571.
[I 2025-10-29 05:51:25,139] Trial 2 finished with value: 25894.074689009452 and parameters: {'n_estimators': 671, 'learning_rate': 0.016411094886272914, 'max_depth': 8, 'subsample': 0.8130233305248951, 'colsample_bytree': 0.6022525554449438}. Best is trial 0 with value: 25518.7422887571.
[I 2025-10-29 05:51:26,

✅ Best Parameters: {'n_estimators': 544, 'learning_rate': 0.030585912019072414, 'max_depth': 5, 'subsample': 0.7159756402998797, 'colsample_bytree': 0.7453699127918348}


[I 2025-10-29 05:51:43,080] Trial 0 finished with value: 27292.73720241339 and parameters: {'n_estimators': 769, 'learning_rate': 0.2663276565473556, 'max_depth': 9, 'subsample': 0.7623366640774738, 'colsample_bytree': 0.6842887721005715}. Best is trial 0 with value: 27292.73720241339.
[I 2025-10-29 05:51:44,416] Trial 1 finished with value: 25315.849264838027 and parameters: {'n_estimators': 589, 'learning_rate': 0.026591761516430234, 'max_depth': 3, 'subsample': 0.9036633266061527, 'colsample_bytree': 0.6376642670896363}. Best is trial 1 with value: 25315.849264838027.
[I 2025-10-29 05:51:45,452] Trial 2 finished with value: 26859.52077011055 and parameters: {'n_estimators': 513, 'learning_rate': 0.05133219820949999, 'max_depth': 4, 'subsample': 0.666508906504606, 'colsample_bytree': 0.7300140190824774}. Best is trial 1 with value: 25315.849264838027.
[I 2025-10-29 05:51:46,791] Trial 3 finished with value: 26862.701576721578 and parameters: {'n_estimators': 411, 'learning_rate': 0.0

✅ Best Parameters: {'n_estimators': 605, 'learning_rate': 0.0330482224811229, 'max_depth': 3, 'subsample': 0.8564394957694603, 'colsample_bytree': 0.6172063792535771}


In [27]:
best_model = XGBRegressor(**study.best_trial.params)
pipe = Pipeline([("pre", preprocessor), ("model", best_model)])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)

preds = pipe.predict(X_val)
rmse = mean_squared_error(y_val, preds) **0.5
print(f"✅ Final Model Trained — RMSE: {rmse:.2f}")

✅ Final Model Trained — RMSE: 24124.26


In [28]:
# --- Save model artifacts
import os
os.makedirs("artifacts", exist_ok=True)
joblib.dump(pipe, "artifacts/model_xgb.pkl")

preproc = pipe.named_steps["pre"]
oh = preproc.named_transformers_["cat"].named_steps["onehot"]
features = list(num_cols) + list(oh.get_feature_names_out(cat_cols))
with open("artifacts/feature_columns.json", "w") as f:
    json.dump(features, f)

print("🎯 Model + features saved in artifacts/")

🎯 Model + features saved in artifacts/


In [31]:
#---- Create streamlit app
app_code = """
import json, joblib, pandas as pd, numpy as np, shap, streamlit as st, matplotlib.pyplot as plt
from xgboost import XGBRegressor

st.set_page_config(page_title="🏠 House Price Predictor + SHAP", layout="wide")
st.title("🏠 House Price Predictor (XGBoost + SHAP)")

@st.cache_resource
def load_artifacts():
    model = joblib.load("artifacts/model_xgb.pkl")
    with open("artifacts/feature_columns.json") as f:
        features = json.load(f)
    return model, features

model, features = load_artifacts()

st.header("Enter Features")
data = {f:0.0 for f in features}
for fld, default in [("OverallQual",6),("GrLivArea",1500),("GarageCars",2),("TotalBsmtSF",800)]:
    if fld in data:
        data[fld] = st.number_input(fld, min_value=0.0, value=float(default), step=10.0)

X = pd.DataFrame([data])

if st.button("Predict"):
    pred = model.predict(X)[0]
    st.success(f"Predicted Sale Price: ${pred:,.2f}")

    explainer = shap.TreeExplainer(model.named_steps["model"])
    shap_values = explainer.shap_values(X)
    st.header("🔍 SHAP Feature Importance")
    fig = plt.figure(figsize=(8,5))
    shap.summary_plot(shap_values, X, plot_type="bar", show=False)
    st.pyplot(fig, clear_figure=True)
"""

# ✅ Save app.py file manually
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ Streamlit app.py file created successfully.")

✅ Streamlit app.py file created successfully.


In [46]:
import streamlit as st

st.set_page_config(page_title="🏠 House Price Predictor + SHAP", layout="wide")
st.title("🏠 House Price Predictor (XGBoost + SHAP)")

@st.cache_resource
def load_artifacts():
    model = joblib.load("artifacts/model_xgb.pkl")
    with open("artifacts/feature_columns.json") as f:
        features = json.load(f)
    return model, features

model, features = load_artifacts()

st.header("Enter Features")
data = {f:0.0 for f in features}
for fld, default in [("OverallQual",6),("GrLivArea",1500),("GarageCars",2),("TotalBsmtSF",800)]:
    if fld in data:
        data[fld] = st.number_input(fld, min_value=0.0, value=float(default), step=10.0)

X = pd.DataFrame([data])

if st.button("Predict"):
    pred = model.predict(X)[0]
    st.success(f"Predicted Sale Price: ${pred:,.2f}")

    try:
    inner_model = model.named_steps["model"] if hasattr(model, "named_steps") else model
    # Transform input if pipeline
    X_transformed = (
        model.named_steps["pre"].transform(df_input)
        if hasattr(model, "named_steps")
        else df_input
    )

    # Convert everything safely to numeric
    X_transformed = pd.DataFrame(X_transformed).apply(pd.to_numeric, errors="coerce").fillna(0)

    explainer = shap.TreeExplainer(inner_model)
    shap_values = explainer.shap_values(X_transformed)

    st.set_option("deprecation.showPyplotGlobalUse", False)
    shap.summary_plot(shap_values, X_transformed, plot_type="bar", show=False)
    st.pyplot(bbox_inches="tight")

except Exception as shap_error:
    st.warning(f"⚠️ SHAP visualization skipped: {shap_error}")explainer = shap.TreeExplainer(model.named_steps["model"])
    

IndentationError: expected an indented block after 'try' statement on line 27 (990768986.py, line 28)

In [34]:
import os
os.system("streamlit run app.py")

-1

In [36]:
def ensure_columns(df, required_cols):
    for col in required_cols:
        if col not in df.columns:
            df[col] = 0
    return df[required_cols]

In [45]:
# Prediction + Explainability
# ------------------------------
if st.button("Predict"):
    if model is None:
        st.error("❌ Model not loaded. Please check artifacts folder.")
    elif df_input is None or df_input.empty:
        st.warning("⚠️ Please upload a CSV or enter values first.")
    else:
        try:
            # Add TotalSF automatically if missing
            if "TotalSF" not in df_input.columns:
                df_input["TotalSF"] = (
                    df_input.get("TotalBsmtSF", 0)
                    + df_input.get("1stFlrSF", 0)
                    + df_input.get("2ndFlrSF", 0)
                )

            preds = model.predict(df_input)
            st.success(f"✅ Predicted Sale Price: ${float(np.median(preds)):.2f}")
            st.write("### Prediction Results")
            st.dataframe(pd.DataFrame({"PredictedPrice": preds}))
            st.write("---")
            st.header("🧠 SHAP Explainability")

            try:
                # Use model directly or from pipeline
                inner_model = model.named_steps["model"] if hasattr(model, "named_steps") else model
                explainer = shap.TreeExplainer(inner_model)
                shap_values = explainer.shap_values(
                    model.named_steps["pre"].transform(df_input)
                    if hasattr(model, "named_steps")
                    else df_input
                )
                st.set_option("deprecation.showPyplotGlobalUse", False)
                shap.summary_plot(
                    shap_values,
                    model.named_steps["pre"].transform(df_input)
                    if hasattr(model, "named_steps")
                    else df_input,
                    plot_type="bar",
                    show=False,
                )
                st.pyplot(bbox_inches="tight")
            except Exception as shap_error:
                st.warning(f"⚠️ SHAP visualization skipped: {shap_error}")

        except Exception as e:
            st.error(f"❌ Prediction failed: {e}")

2025-10-29 06:59:27.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-29 06:59:27.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-29 06:59:27.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-29 06:59:27.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-29 06:59:27.507 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
